In [ ]:
# 02-Model-Training.ipynb

import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from pyspark.sql import SparkSession

# डेल्टा टेबल से डेटा लोड करना
delta_table_path = "/tmp/house_price_prediction_data"
df = spark.read.format("delta").load(delta_table_path).toPandas()

X = df[['sq_feet', 'num_bedrooms', 'num_bathrooms', 'year_built', 'location_score']]
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# MLflow का उपयोग करके ट्रेनिंग रन शुरू करना
mlflow.set_experiment("/mlops_house_price_prediction_experiment")

with mlflow.start_run() as run:
    # मॉडल को परिभाषित करना और ट्रेन करना
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # प्रेडिक्शन करना
    predictions = model.predict(X_test)

    # मेट्रिक्स की गणना करना
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    # MLflow के साथ मेट्रिक्स और पैरामीटर को लॉग करना
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2_score", r2)
    
    # मॉडल को MLflow में लॉग करना
    mlflow.sklearn.log_model(model, "house_price_model")
    
    # मॉडल को मॉडल रजिस्ट्री में रजिस्टर करना
    model_name = "HousePricePredictionModel"
    mlflow.register_model(
        f"runs:/{run.info.run_uuid}/house_price_model",
        model_name
    )

print("मॉडल सफलतापूर्वक ट्रेन, लॉग, और मॉडल रजिस्ट्री में रजिस्टर हो गया है।")